In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_hub as hub
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# In this notebook I will try to keep things very simple
To under stand this notebook, you don't need to know much about RNN, LSTM RNN ,etc... or any advanced techniques
As we no the dataset does not contain really long paragraphs,
We will just use google universal sentence encoder<br>
It takes a sentence as inputs and returns a 512 dimensional vector <br>
These vectors are capable of mathematically capturing basic meaning of the sentence in 512 dimensions<br>
Read more : [https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder](http://)<br>
After we get that 512 dimensional vector
we will pass it through a simple ann for classifying it into 3 categories

In [ ]:

# Check if GPU is available
if tf.test.is_gpu_available():
    print("GPU is available")
    print("GPU devices:")
    for device in tf.config.experimental.list_physical_devices('GPU'):
        print(device)
else:
    print("GPU is not available")

# TensorFlow operations will automatically run on GPU if available


In [ ]:
df = pd.read_csv("/kaggle/input/twitter-sentiment-dataset/Twitter_Data.csv")

In [ ]:
df

## let's try training on small dataset

In [ ]:
dataset = df[0:10000]

from sklearn.model_selection import train_test_split

train_df,test_df =  train_test_split(dataset,test_size=0.2)

train_df = train_df.reset_index(drop=True).dropna()
val_df = test_df.reset_index(drop=True).dropna()

x_train,y_train = train_df.clean_text.astype(str),train_df.category
x_val,y_val = val_df.clean_text.astype(str),val_df.category
y_train+= 1
y_val+=1


In [ ]:
dataset.category.hist()

In [ ]:
train_df.category.hist()

In [ ]:
val_df.category.hist()

In [ ]:
# the dataset is fairly balanced at least we don't need to balance it ourselves

In [ ]:
y_val.value_counts(),y_train.value_counts()

# Using google universal sentence encoder

## google sentence encoder is used to transform sentences intor a meaningful vector of 512 dimensions

In [ ]:
sentence_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
trainable=True, dtype=tf.string, input_shape=[])

In [ ]:
# using dropout and l2 reg. for avoiding overfitting
feed_forward_net = tf.keras.Sequential([
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1024,activation='leaky_relu',kernel_regularizer = tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(1024,activation='leaky_relu',kernel_regularizer = tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(3,activation='softmax')
])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[],dtype='string'),
    sentence_encoder,
    
    feed_forward_net
])

In [ ]:
model(x_train[0:10])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(),
             metrics = ['accuracy'])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model_checkpoint.h5',
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         mode='max',
                                                         save_best_only=True)

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=10,callbacks=[checkpoint_callback],validation_data = (x_val,y_val))

In [ ]:
model.evaluate(x_val,y_val)

In [ ]:
model.load_weights("model_checkpoint.h5")

In [ ]:
model.evaluate(x_val,y_val)

In [ ]:
test_x, test_y = df[20000:30000].clean_text, df[20000:30000].category+1

In [ ]:
model.evaluate(test_x,test_y)

So we are getting good accuracy after training on small dataset, Now let's train on more data

# Training on large dataset

In [ ]:
len(df)

In [ ]:

train_val_dataset = df[0:-20000]


from sklearn.model_selection import train_test_split

train_df,test_df =  train_test_split(train_val_dataset,test_size=0.2)

train_df = train_df.reset_index(drop=True).dropna()
val_df = test_df.reset_index(drop=True).dropna()

x_train,y_train = train_df.clean_text.astype(str),train_df.category
x_val,y_val = val_df.clean_text.astype(str),val_df.category
y_train+= 1
y_val+=1


In [ ]:
model(test_x[0:5])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = tf.keras.optimizers.Adam(),
             metrics = ['accuracy'])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='model_large_dataset_checkpoint.h5',
                                                         save_weights_only=True,
                                                         monitor='val_accuracy',
                                                         mode='max',
                                                         save_best_only=True)

In [ ]:
model.fit(x_train,y_train,batch_size=32,epochs=10,callbacks=[checkpoint_callback],validation_data = (x_val,y_val))

In [ ]:
model.load_weights("model_large_dataset_checkpoint.h5")

In [ ]:
model.evaluate(x_val,y_val)

### Finally we have got 88% validation accuracy after training on large dataset, which is quite good and a big improvement from 75% (on small training dataset)